# Seperate Neural Network Tuning

- Using GPU


## Set Up

Packages + libraries + proj globals

In [ ]:
import sys
import os
import subprocess

sys.path.append("../")
from src.config import SEED, BASE_PATH, DEVICE, NN_TUNE_STAGE

print(f"Using device: {DEVICE}")
print(f"Path: {BASE_PATH}")
print(f"STAGE: {NN_TUNE_STAGE}")

Set notebook globals

In [ ]:
import_path = BASE_PATH / "data" / "processed"
outcome_abrvs = {}
for outcome_dir in import_path.iterdir():
    ## For the outcome_df.parquet file
    if outcome_dir.is_file():
        continue
    outcome_name = outcome_dir.name
    outcome_abrv = outcome_name.split("_")[0]
    outcome_abrvs[outcome_dir.name] = outcome_abrv

In [ ]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(i, torch.cuda.get_device_name(i))

## Build + Tune models

In [ ]:
env = os.environ.copy()
env["PYTHONPATH"] = str(BASE_PATH)
script_path = BASE_PATH / "src" / "tune_nn.py"
match NN_TUNE_STAGE:
    case 1:
        n_trials = 150
    case 2:
        n_trials = 100
    case 3:
        n_trials = 100  # can raise or lower depending on stage 2
    case _:
        raise ValueError(
            f"STAGE in src/config.py must be one of [1,2,3]. Got {NN_TUNE_STAGE} instead."
        )
cmds = []
for gpu_idx, (outcome_name, outcome_abrv) in enumerate(outcome_abrvs.items()):
    cmds.append(
        [
            "uv",
            "run",
            str(script_path),
            "--outcome_name",
            outcome_name,
            "--X_path",
            str(BASE_PATH / "data" / "processed" / outcome_name / "X_train.parquet"),
            "--y_path",
            str(BASE_PATH / "data" / "processed" / outcome_name / "y_train.xlsx"),
            "--gpu_ids_str",
            str(gpu_idx),  ## each tuner gets 1 GPU
            "--scoring_str",
            "roc_auc",
            "--log_path",
            str(BASE_PATH / "tune_results" / "logs" / "nn3" / f"{outcome_abrv}.log"),
            "--results_path",
            str(
                BASE_PATH
                / "tune_results"
                / "best_params"
                / "nn3"
                / f"{outcome_abrv}.json"
            ),
            "--n_trials",
            str(n_trials),
            "--seed",
            str(SEED),
        ]
    )

Run commands
- only run once

In [ ]:
procs = [subprocess.Popen(cmd, env=env) for cmd in cmds]
[p.poll() for p in procs]

Monitor commands 
- None means still running, otherwise provides exit code
    - 0 indicates a successful exit
- Can run as many times as u want

In [ ]:
[p.poll() for p in procs]

Kill processes
- Only run once
- should output non-zero exit codes for each process
    - if not, run above cell again to monitor

In [ ]:
for p in procs:
    p.terminate()
[p.poll() for p in procs]

## Train models + prelim results

- Use CPU for platform portability later

In [ ]:
from src.data_utils import get_data
import json
from src.tune_nn import train_and_prelim_eval

file_dir = BASE_PATH / "data" / "processed"

OUTCOME_DICT = {
    "med": get_data("med_outcome", file_dir),
    "surg": get_data("surg_outcome", file_dir),
    "mort": get_data("mort_outcome", file_dir),
    "reop": get_data("reop_outcome", file_dir),
    "vte": get_data("vte_outcome", file_dir),
}

In [ ]:
for outcome_name, outcome_data in OUTCOME_DICT.items():
    train_and_prelim_eval(
        outcome_name=outcome_name,
        data_dict=OUTCOME_DICT[outcome_name],
        json_path=BASE_PATH
        / "models"
        / "tune_results_3"
        / "nn"
        / f"{outcome_name}.json",
        model_save_path=BASE_PATH / "models" / "trained" / outcome_name / "nn.pt",
    )